In [10]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [2]:
# Define feature extraction function
def extract_mfcc_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs.T, axis=0)

In [3]:
# Define data loading function
def load_data(data_dir, n_classes=9, n_mfcc=13):
    features = []
    labels = []
    for label in range(n_classes):
        class_dir = os.path.join(data_dir, str(label))
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            mfccs = extract_mfcc_features(file_path, n_mfcc)
            features.append(mfccs)
            labels.append(label)
    return np.array(features), np.array(labels)

In [4]:
# Load dataset and extract features
data_dir = 'Datasets/Augment_KNN/'  # Path to your dataset directory
n_classes = 10  # Number of classes (9 classes of gunshots & 1 classes of non-gunshots)
n_mfcc = 13  # Number of MFCC features

features, labels = load_data(data_dir, n_classes, n_mfcc)

In [5]:
# Reshape features for RNN input
features = features.reshape(features.shape[0], features.shape[1], 1)

# Convert labels to categorical
labels = to_categorical(labels, num_classes=n_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [6]:
# Build the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(n_mfcc, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))  # For multi-class classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/Users/shazanadiatulhudamohdsalleh/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 13, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,378 (462.41 KB)

 Trainable params: 118,378 (462.41 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.2580 - loss: 2.0162 - val_accuracy: 0.4638 - val_loss: 1.5069
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4947 - loss: 1.4379 - val_accuracy: 0.5500 - val_loss: 1.2746
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5615 - loss: 1.2206 - val_accuracy: 0.6139 - val_loss: 1.0599
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6004 - loss: 1.0996 - val_accuracy: 0.6515 - val_loss: 0.9140
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6528 - loss: 0.9184 - val_accuracy: 0.6671 - val_loss: 0.8855
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6703 - loss: 0.8735 - val_accuracy: 0.6930 - val_loss: 0.7959
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6861 - loss: 0.8071 - val_accuracy: 0.7176 - val_loss: 0.7482
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7048 - loss: 0.7456 - val_accu

In [8]:
# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_class = np.argmax(y_test, axis=1)

print(classification_report(y_test_class, y_pred))

70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       223
           1       1.00      1.00      1.00       223
           2       0.92      0.96      0.94       220
           3       0.48      0.86      0.62       226
           4       0.31      0.06      0.10       211
           5       0.96      0.92      0.94       220
           6       0.99      0.97      0.98       216
           7       0.91      0.94      0.92       205
           8       1.00      0.98      0.99       228
           9       0.98      0.93      0.95       266

    accuracy                           0.87      2238
   macro avg       0.85      0.86      0.84      2238
weighted avg       0.86      0.87      0.85      2238



In [11]:
accuracy = accuracy_score(y_test_class, y_pred)
precision = precision_score(y_test_class, y_pred, average='weighted')
recall = recall_score(y_test_class, y_pred, average='weighted')
f1 = f1_score(y_test_class, y_pred, average='weighted')

In [12]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

print("Classification Report:")
print(classification_report(y_test_class, y_pred))

Accuracy: 0.8663985701519213
Precision: 0.857046511959558
Recall: 0.8663985701519213
F1 Score: 0.8481333921232519
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       223
           1       1.00      1.00      1.00       223
           2       0.92      0.96      0.94       220
           3       0.48      0.86      0.62       226
           4       0.31      0.06      0.10       211
           5       0.96      0.92      0.94       220
           6       0.99      0.97      0.98       216
           7       0.91      0.94      0.92       205
           8       1.00      0.98      0.99       228
           9       0.98      0.93      0.95       266

    accuracy                           0.87      2238
   macro avg       0.85      0.86      0.84      2238
weighted avg       0.86      0.87      0.85      2238

